In [57]:
import pandas as pd
import requests
import sqlite3

In [2]:
# ===================================================================
# PASO 1: EXTRACCIÓN (EXTRACT) DE LOS DATOS
# ===================================================================

# La plataforma datos.gov.co usa la API de Socrata. Podemos usarla para
# descargar los datos directamente.
api_url = "https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000"

print(f"📥 Extrayendo datos desde: {api_url}")

try:
    response = requests.get(api_url)
    response.raise_for_status()  # Lanza un error si la petición falla
    data = response.json()
    df_raw = pd.DataFrame(data)
    print(f"✅ ¡Extracción exitosa! Se cargaron {len(df_raw)} filas.")
    display(df_raw.head())

except requests.exceptions.RequestException as e:
    print(f"❌ Error al extraer los datos: {e}")
    df_raw = pd.DataFrame()  # Creamos un DataFrame vacío para evitar errores posteriores

except Exception as e:
    print(f"❌ Ocurrió un error inesperado: {e}")
    df_raw = pd.DataFrame()


📥 Extrayendo datos desde: https://www.datos.gov.co/resource/nudc-7mev.json?$limit=50000
✅ ¡Extracción exitosa! Se cargaron 14585 filas.


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,c_digo_etc,etc,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,...,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,3758,Antioquia (ETC),503,62.62,62.62,...,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,3830,Guaviare (ETC),4438,53.27,53.27,...,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,3830,Guaviare (ETC),2014,32.52,32.52,...,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,3831,Vaupés (ETC),10986,59.57,59.57,...,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,3831,Vaupés (ETC),1228,51.3,51.3,...,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [8]:
# Mostrar los departamentos únicos en formato de tabla
df_raw['departamento'].unique()


array(['Antioquia', 'Guaviare', 'Vaupés', 'Vichada', 'Guainía',
       'Amazonas',
       'Archipiélago de San Andrés, Providencia y Santa Catalina',
       'Putumayo', 'Casanare', 'Arauca', 'Valle del Cauca', 'Tolima',
       'Sucre', 'Santander', 'Risaralda', 'Quindio', 'Norte de Santander',
       'Nariño', 'Meta', 'Magdalena', 'La Guajira', 'Huila', 'Chocó',
       'Cundinamarca', 'Córdoba', 'Cesar', 'Cauca', 'Caquetá', 'Caldas',
       'Boyacá', 'Bolívar', 'Bogotá, D.C.', 'Atlántico', 'NACIONAL',
       'Archipiélago de San Andrés. Providencia y Santa Catalina',
       'Bogotá D.C.'], dtype=object)

In [12]:
# Reemplazar los valores inconsistentes y normalizar los nombres de los departamentos
df_raw['departamento'] = df_raw['departamento'].replace({
    'Bogotá, D.C.': 'Bogotá D.C',  # Consolidar Bogotá
    'Bogotá D.C.': 'Bogotá D.C',  # Asegurarnos de que no haya variantes
    'Archipiélago de San Andrés. Providencia y Santa Catalina': 'Archipiélago de San Andrés, Providencia y Santa Catalina',  # Unificar nombre completo
    'Archipiélago de San Andrés, Providencia y Santa Catalina': 'Archipiélago de San Andrés, Providencia y Santa Catalina',  # Asegurar que ambos sean iguales
    'NACIONAL': 'No Aplica',  
})

In [13]:
# departamentos únicos después de la limpieza
departamentos_unicos = pd.DataFrame(df_raw['departamento'].unique(), columns=['Departamento'])
display(departamentos_unicos)

,Departamento
0,Antioquia
1,Guaviare
2,Vaupés
3,Vichada
4,Guainía
5,Amazonas
6,"Archipiélago de San Andrés, Providencia y Sant..."
7,Putumayo
8,Casanare
9,Arauca


In [14]:
df_raw = df_raw[df_raw['departamento'] != 'No Aplica']

departamentos_unicos = pd.DataFrame(df_raw['departamento'].unique(), columns=['Departamento'])
display(departamentos_unicos)

,Departamento
0,Antioquia
1,Guaviare
2,Vaupés
3,Vichada
4,Guainía
5,Amazonas
6,"Archipiélago de San Andrés, Providencia y Sant..."
7,Putumayo
8,Casanare
9,Arauca


In [16]:
df_raw.columns

Index(['a_o', 'c_digo_municipio', 'municipio', 'c_digo_departamento',
       'departamento', 'c_digo_etc', 'etc', 'poblaci_n_5_16',
       'tasa_matriculaci_n_5_16', 'cobertura_neta',
       'cobertura_neta_transici_n', 'cobertura_neta_primaria',
       'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta',
       'cobertura_bruta_transici_n', 'cobertura_bruta_primaria',
       'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n',
       'deserci_n_transici_n', 'deserci_n_primaria', 'deserci_n_secundaria',
       'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n',
       'aprobaci_n_primaria', 'aprobaci_n_secundaria', 'aprobaci_n_media',
       'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria',
       'reprobaci_n_secundaria', 'reprobaci_n_media', 'repitencia',
       'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria',
       'repitencia_media', 'tama_o_promedio_de_grupo',
       'sedes_conectadas_a_internet'],
   

In [ ]:
# eliminar etc
pd.set_option('display.max_columns', None)

# df_raw.drop(columns=['etc'], inplace=True) ya se elimino

In [22]:
df_raw.drop(columns=['c_digo_etc'], inplace=True) 

In [23]:
display(df_raw.head())

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,503,62.62,62.62,44.19,63.33,51.53,40.23,66.8,58.14,72.86,66.87,56.32,1.19,0,1.31,0,4.08,92.26,0,96.73,83.49,93.88,6.55,0,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,4438,53.27,53.27,33.91,48.89,44.9,21.3,62.98,54.2,65.19,69.6,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0,87.9,84.5,92.98,6.78,0,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,2014,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.1,26.16,7.85,15,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.5,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,10986,59.57,59.57,42.76,55.95,43.51,17.06,70.65,64.9,76.96,72.92,53.12,3.95,2.27,1.84,6.77,5.47,90.71,0.57,94.12,84.91,89.93,5.34,0.57,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,1228,51.3,51.3,76.32,52.29,33.71,11.94,55.54,92.11,65.21,51.12,27.36,8.36,4.29,3.05,15.72,14.55,82.4,0,89.63,69,78.18,9.24,0,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [24]:
# Función para validar si los valores están dentro de un rango permitido (0 a 100)
def validate_range(value):
    if isinstance(value, (int, float)):
        if 0 <= value <= 100:
            return value
        else:
            return None  # Si está fuera de rango, lo convertimos en NaN
    return value

# Aplicamos esta validación a todas las columnas numéricas
columnas_numericas = [
    'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_neta_transici_n', 'cobertura_neta_primaria',
    'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n', 'cobertura_bruta_primaria',
    'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria',
    'deserci_n_secundaria', 'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria', 'aprobaci_n_secundaria',
    'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media',
    'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria', 'repitencia_media', 'tama_o_promedio_de_grupo',
    'sedes_conectadas_a_internet'
]

# Aplicamos la validación a cada columna numérica
for col in columnas_numericas:
    df_raw[col] = df_raw[col].apply(validate_range)

# Mostrar las primeras filas después de la validación
display(df_raw.head())


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,503,62.62,62.62,44.19,63.33,51.53,40.23,66.8,58.14,72.86,66.87,56.32,1.19,0,1.31,0,4.08,92.26,0,96.73,83.49,93.88,6.55,0,1.96,16.51,2.04,9.52,0,10.46,13.76,2.04,None,None
1,2023,95025,El Retorno,95,Guaviare,4438,53.27,53.27,33.91,48.89,44.9,21.3,62.98,54.2,65.19,69.6,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0,87.9,84.5,92.98,6.78,0,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,None,None
2,2023,95200,Miraflores,95,Guaviare,2014,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.1,26.16,7.85,15,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.5,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,None,None
3,2023,97001,Mitú,97,Vaupés,10986,59.57,59.57,42.76,55.95,43.51,17.06,70.65,64.9,76.96,72.92,53.12,3.95,2.27,1.84,6.77,5.47,90.71,0.57,94.12,84.91,89.93,5.34,0.57,4.04,8.33,4.6,16.18,7.75,21.04,13.84,7.18,None,None
4,2023,97161,Caruru,97,Vaupés,1228,51.3,51.3,76.32,52.29,33.71,11.94,55.54,92.11,65.21,51.12,27.36,8.36,4.29,3.05,15.72,14.55,82.4,0,89.63,69,78.18,9.24,0,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,None,None


In [ ]:
# Convertirtipo float 
df_raw[columnas_numericas] = df_raw[columnas_numericas].apply(pd.to_numeric, errors='coerce')

display(df_raw.head())

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,503.0,62.62,62.62,44.19,63.33,51.53,40.23,66.80,58.14,72.86,66.87,56.32,1.19,0.00,1.31,0.00,4.08,92.26,0.00,96.73,83.49,93.88,6.55,0.00,1.96,16.51,2.04,9.52,0.00,10.46,13.76,2.04,NaN,NaN
1,2023,95025,El Retorno,95,Guaviare,4438.0,53.27,53.27,33.91,48.89,44.90,21.30,62.98,54.20,65.19,69.60,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0.00,87.90,84.50,92.98,6.78,0.00,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,NaN,NaN
2,2023,95200,Miraflores,95,Guaviare,2014.0,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.10,26.16,7.85,15.00,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.50,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,NaN,NaN
3,2023,97001,Mitú,97,Vaupés,10986.0,59.57,59.57,42.76,55.95,43.51,17.06,70.65,64.90,76.96,72.92,53.12,3.95,2.27,1.84,6.77,5.47,90.71,0.57,94.12,84.91,89.93,5.34,0.57,4.04,8.33,4.60,16.18,7.75,21.04,13.84,7.18,NaN,NaN
4,2023,97161,Caruru,97,Vaupés,1228.0,51.30,51.30,76.32,52.29,33.71,11.94,55.54,92.11,65.21,51.12,27.36,8.36,4.29,3.05,15.72,14.55,82.40,0.00,89.63,69.00,78.18,9.24,0.00,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,NaN,NaN


In [27]:
# valores nulos con la mediana de cada columna numérica
for col in columnas_numericas:
    df_raw[col].fillna(df_raw[col].median(), inplace=True)


display(df_raw.head())

C:\Users\laura\AppData\Local\Temp\ipykernel_15576\2565576813.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_raw[col].fillna(df_raw[col].median(), inplace=True)
C:\Users\laura\AppData\Local\Temp\ipykernel_15576\2565576813.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

F

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023,05004,Abriaquí,05,Antioquia,503.0,62.62,62.62,44.19,63.33,51.53,40.23,66.80,58.14,72.86,66.87,56.32,1.19,0.00,1.31,0.00,4.08,92.26,0.00,96.73,83.49,93.88,6.55,0.00,1.96,16.51,2.04,9.52,0.00,10.46,13.76,2.04,23.96,28.13
1,2023,95025,El Retorno,95,Guaviare,4438.0,53.27,53.27,33.91,48.89,44.90,21.30,62.98,54.20,65.19,69.60,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0.00,87.90,84.50,92.98,6.78,0.00,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,23.96,28.13
2,2023,95200,Miraflores,95,Guaviare,2014.0,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.10,26.16,7.85,15.00,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.50,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,23.96,28.13
3,2023,97001,Mitú,97,Vaupés,10986.0,59.57,59.57,42.76,55.95,43.51,17.06,70.65,64.90,76.96,72.92,53.12,3.95,2.27,1.84,6.77,5.47,90.71,0.57,94.12,84.91,89.93,5.34,0.57,4.04,8.33,4.60,16.18,7.75,21.04,13.84,7.18,23.96,28.13
4,2023,97161,Caruru,97,Vaupés,1228.0,51.30,51.30,76.32,52.29,33.71,11.94,55.54,92.11,65.21,51.12,27.36,8.36,4.29,3.05,15.72,14.55,82.40,0.00,89.63,69.00,78.18,9.24,0.00,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,23.96,28.13


In [28]:
# Valores nulos prueba
df_raw.isnull().sum().sort_values(ascending=False)

a_o                            0
c_digo_municipio               0
municipio                      0
c_digo_departamento            0
departamento                   0
poblaci_n_5_16                 0
tasa_matriculaci_n_5_16        0
cobertura_neta                 0
cobertura_neta_transici_n      0
cobertura_neta_primaria        0
cobertura_neta_secundaria      0
cobertura_neta_media           0
cobertura_bruta                0
cobertura_bruta_transici_n     0
cobertura_bruta_primaria       0
cobertura_bruta_secundaria     0
cobertura_bruta_media          0
deserci_n                      0
deserci_n_transici_n           0
deserci_n_primaria             0
deserci_n_secundaria           0
deserci_n_media                0
aprobaci_n                     0
aprobaci_n_transici_n          0
aprobaci_n_primaria            0
aprobaci_n_secundaria          0
aprobaci_n_media               0
reprobaci_n                    0
reprobaci_n_transici_n         0
reprobaci_n_primaria           0
reprobaci_

In [29]:
df_raw.describe()

,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
count,1.458200e+04,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000,14582.000000
mean,8.221956e+03,84.992045,85.577053,57.704857,82.585241,69.926184,40.758024,97.579165,87.135333,105.093967,101.135100,75.865937,3.487122,3.457339,2.749975,4.568885,3.435997,91.627448,0.483803,93.233581,88.379478,92.167227,4.756870,0.483803,3.881868,6.805669,4.121498,3.288975,0.933135,3.157061,4.310828,1.690343,23.955956,31.699926
std,4.452569e+04,18.478310,16.825936,16.150646,17.111552,18.746043,15.570022,21.208524,25.453826,25.174776,25.708737,26.856202,2.161911,2.688775,2.031311,3.074556,3.190088,5.162328,1.648629,5.041247,7.858355,6.867764,3.987597,1.648629,3.790051,6.131181,4.192027,3.355109,2.404131,3.627971,4.527894,2.274741,5.551520,19.400318
min,1.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,0.000000
25%,1.165000e+03,74.950000,76.962500,47.590000,73.310000,60.500000,31.192500,85.730000,71.560000,89.910000,86.642500,59.760000,1.950000,1.630000,1.280000,2.380000,1.690000,88.450000,0.000000,90.580000,83.660000,89.470000,0.950000,0.000000,0.430000,0.762500,0.682500,0.730000,0.000000,0.600000,0.750000,0.000000,23.680000,25.810000
50%,2.639000e+03,85.330000,86.400000,57.900000,82.840000,70.900000,41.180000,97.490000,85.220000,103.320000,101.535000,75.250000,3.130000,3.000000,2.320000,4.040000,2.850000,92.020000,0.000000,93.870000,89.100000,93.290000,4.520000,0.000000,3.330000,6.160000,3.110000,2.160000,0.000000,1.880000,2.740000,0.870000,23.960000,28.130000
75%,5.862000e+03,95.307500,94.477500,68.000000,91.867500,80.500000,50.500000,109.190000,100.000000,117.600000,115.630000,90.240000,4.640000,4.710000,3.750000,6.150000,4.400000,95.580000,0.210000,96.940000,94.440000,96.410000,7.397500,0.210000,5.980000,10.790000,6.360000,5.047500,0.920000,4.550000,6.770000,2.530000,24.250000,30.417500
max,1.479334e+06,279.030000,264.540000,150.470000,254.960000,229.440000,170.260000,286.920000,259.290000,326.400000,301.710000,489.620000,27.900000,42.860000,18.310000,47.140000,62.500000,100.000000,52.710000,100.000000,100.000000,100.000000,49.390000,52.710000,51.970000,76.970000,67.860000,37.470000,50.000000,50.760000,55.070000,36.450000,54.030000,100.000000


La cobertura educativa y la tasa de matrícula pueden exceder el 100% debido a fenómenos como la extraedad (cuando los estudiantes están fuera del rango de edad teórica para un nivel educativo) y los flujos migratorios no reflejados en las proyecciones poblacionales. Este comportamiento es frecuente en regiones con alta demanda social o movimientos migratorios significativos.

In [32]:
# Verificar y convertir columnas con fechas
df_raw['a_o'] = pd.to_datetime(df_raw['a_o'], errors='coerce')

display(df_raw.head())


,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023-01-01,05004,Abriaquí,05,Antioquia,503.0,62.62,62.62,44.19,63.33,51.53,40.23,66.80,58.14,72.86,66.87,56.32,1.19,0.00,1.31,0.00,4.08,92.26,0.00,96.73,83.49,93.88,6.55,0.00,1.96,16.51,2.04,9.52,0.00,10.46,13.76,2.04,23.96,28.13
1,2023-01-01,95025,El Retorno,95,Guaviare,4438.0,53.27,53.27,33.91,48.89,44.90,21.30,62.98,54.20,65.19,69.60,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0.00,87.90,84.50,92.98,6.78,0.00,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,23.96,28.13
2,2023-01-01,95200,Miraflores,95,Guaviare,2014.0,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.10,26.16,7.85,15.00,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.50,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,23.96,28.13
3,2023-01-01,97001,Mitú,97,Vaupés,10986.0,59.57,59.57,42.76,55.95,43.51,17.06,70.65,64.90,76.96,72.92,53.12,3.95,2.27,1.84,6.77,5.47,90.71,0.57,94.12,84.91,89.93,5.34,0.57,4.04,8.33,4.60,16.18,7.75,21.04,13.84,7.18,23.96,28.13
4,2023-01-01,97161,Caruru,97,Vaupés,1228.0,51.30,51.30,76.32,52.29,33.71,11.94,55.54,92.11,65.21,51.12,27.36,8.36,4.29,3.05,15.72,14.55,82.40,0.00,89.63,69.00,78.18,9.24,0.00,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,23.96,28.13


In [33]:
#categorías únicas en la columna 'departamento'
departamentos_unicos = df_raw['departamento'].unique()
print(departamentos_unicos)

['Antioquia' 'Guaviare' 'Vaupés' 'Vichada' 'Guainía' 'Amazonas'
 'Archipiélago de San Andrés, Providencia y Santa Catalina' 'Putumayo'
 'Casanare' 'Arauca' 'Valle del Cauca' 'Tolima' 'Sucre' 'Santander'
 'Risaralda' 'Quindio' 'Norte de Santander' 'Nariño' 'Meta' 'Magdalena'
 'La Guajira' 'Huila' 'Chocó' 'Cundinamarca' 'Córdoba' 'Cesar' 'Cauca'
 'Caquetá' 'Caldas' 'Boyacá' 'Bolívar' 'Bogotá D.C' 'Atlántico']


In [37]:
# verificar las categorías únicas en la columna 'municipio'
df_raw['municipio'].unique()


array(['Abriaquí', 'El Retorno', 'Miraflores', ..., 'Alejandría',
       'Bogotá D.C.', 'Mapiripana'], shape=(1036,), dtype=object)

In [38]:
# Agrupar por 'c_digo_municipio' y contar cuántos nombres de 'municipio' diferentes hay
municipios_duplicados = df_raw.groupby('c_digo_municipio')['municipio'].nunique()

# Filtrar los municipios que tienen más de un nombre asociado
municipios_con_duplicados = municipios_duplicados[municipios_duplicados > 1]

# Mostrar los municipios con duplicados
print("Municipios con duplicados (más de un nombre asociado):")
print(municipios_con_duplicados)


Municipios con duplicados (más de un nombre asociado):
c_digo_municipio
11001    2
Name: municipio, dtype: int64


In [41]:
municipios_duplicados_11001 = df_raw[df_raw['c_digo_municipio'] == '11001']
display(municipios_duplicados_11001[['c_digo_municipio', 'municipio']])

,c_digo_municipio,municipio
974,11001,"Bogotá, D.C."
2088,11001,"Bogotá, D.C."
3215,11001,"Bogotá, D.C."
3653,11001,Bogotá D.C.
5457,11001,"Bogotá, D.C."
6570,11001,"Bogotá, D.C."
7709,11001,"Bogotá, D.C."
8007,11001,"Bogotá, D.C."
9941,11001,"Bogotá, D.C."
11067,11001,"Bogotá, D.C."


In [43]:
bogota_correccion = df_raw[df_raw['municipio'] == 'Bogotá, D.C.']
display(bogota_correccion[['c_digo_municipio', 'municipio']])


,c_digo_municipio,municipio
974,11001,"Bogotá, D.C."
2088,11001,"Bogotá, D.C."
3215,11001,"Bogotá, D.C."
3653,11001,"Bogotá, D.C."
5457,11001,"Bogotá, D.C."
6570,11001,"Bogotá, D.C."
7709,11001,"Bogotá, D.C."
8007,11001,"Bogotá, D.C."
9941,11001,"Bogotá, D.C."
11067,11001,"Bogotá, D.C."


## Convertir Categoria

In [44]:
df_raw['departamento'] = df_raw['departamento'].astype('category')
df_raw['municipio'] = df_raw['municipio'].astype('category')

# Verificar el tipo de las columnas después de la conversión
print(df_raw.dtypes)


a_o                            datetime64[ns]
c_digo_municipio                       object
municipio                            category
c_digo_departamento                    object
departamento                         category
poblaci_n_5_16                        float64
tasa_matriculaci_n_5_16               float64
cobertura_neta                        float64
cobertura_neta_transici_n             float64
cobertura_neta_primaria               float64
cobertura_neta_secundaria             float64
cobertura_neta_media                  float64
cobertura_bruta                       float64
cobertura_bruta_transici_n            float64
cobertura_bruta_primaria              float64
cobertura_bruta_secundaria            float64
cobertura_bruta_media                 float64
deserci_n                             float64
deserci_n_transici_n                  float64
deserci_n_primaria                    float64
deserci_n_secundaria                  float64
deserci_n_media                   

In [49]:
df_raw.head(3)

,a_o,c_digo_municipio,municipio,c_digo_departamento,departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,2023-01-01,05004,Abriaquí,05,Antioquia,503.0,62.62,62.62,44.19,63.33,51.53,40.23,66.80,58.14,72.86,66.87,56.32,1.19,0.00,1.31,0.00,4.08,92.26,0.00,96.73,83.49,93.88,6.55,0.00,1.96,16.51,2.04,9.52,0.00,10.46,13.76,2.04,23.96,28.13
1,2023-01-01,95025,El Retorno,95,Guaviare,4438.0,53.27,53.27,33.91,48.89,44.90,21.30,62.98,54.20,65.19,69.60,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0.00,87.90,84.50,92.98,6.78,0.00,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,23.96,28.13
2,2023-01-01,95200,Miraflores,95,Guaviare,2014.0,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.10,26.16,7.85,15.00,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.50,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,23.96,28.13


In [54]:
df_raw['tasa_matriculaci_n_5_16'] = pd.to_numeric(df_raw['tasa_matriculaci_n_5_16'], errors='coerce')
df_raw['cobertura_neta'] = pd.to_numeric(df_raw['cobertura_neta'], errors='coerce')
df_raw['a_o'] = pd.to_datetime(df_raw['a_o'], format='%Y', errors='coerce')

In [50]:
columnas_numericas = [
    'poblaci_n_5_16', 'tasa_matriculaci_n_5_16', 'cobertura_neta', 'cobertura_neta_transici_n', 'cobertura_neta_primaria',
    'cobertura_neta_secundaria', 'cobertura_neta_media', 'cobertura_bruta', 'cobertura_bruta_transici_n', 'cobertura_bruta_primaria',
    'cobertura_bruta_secundaria', 'cobertura_bruta_media', 'deserci_n', 'deserci_n_transici_n', 'deserci_n_primaria',
    'deserci_n_secundaria', 'deserci_n_media', 'aprobaci_n', 'aprobaci_n_transici_n', 'aprobaci_n_primaria', 'aprobaci_n_secundaria',
    'aprobaci_n_media', 'reprobaci_n', 'reprobaci_n_transici_n', 'reprobaci_n_primaria', 'reprobaci_n_secundaria', 'reprobaci_n_media',
    'repitencia', 'repitencia_transici_n', 'repitencia_primaria', 'repitencia_secundaria', 'repitencia_media', 'tama_o_promedio_de_grupo',
    'sedes_conectadas_a_internet'
]

In [ ]:
# Dimensión Municipio 
dim_municipio = df_raw[['c_digo_municipio', 'municipio']]
dim_municipio['id_municipio'] = range(1, len(dim_municipio) + 1)  

# Dimensión Departamento 
dim_departamento = df_raw[['c_digo_departamento', 'departamento']]
dim_departamento['id_departamento'] = range(1, len(dim_departamento) + 1)  

# Verificar las dimensiones creadas
display(dim_municipio.head()) 
display(dim_departamento.head())  

,c_digo_municipio,municipio,id_municipio
0,05004,Abriaquí,1
1,95025,El Retorno,2
2,95200,Miraflores,3
3,97001,Mitú,4
4,97161,Caruru,5


,c_digo_departamento,departamento,id_departamento
0,05,Antioquia,1
1,95,Guaviare,2
3,97,Vaupés,3
9,99,Vichada,4
14,94,Guainía,5


## Tabla de hechos

In [64]:
# tabla de hechos 
df_hechos = df_raw[['c_digo_municipio', 'c_digo_departamento', 'a_o'] + columnas_numericas].copy()

# Unir la tabla de hechos con la dimensión Municipio para obtener 'id_municipio'
df_hechos = df_hechos.merge(dim_municipio[['c_digo_municipio', 'id_municipio']], on='c_digo_municipio', how='left')

# Unir la tabla de hechos con la dimensión Departamento para obtener 'id_departamento'
df_hechos = df_hechos.merge(dim_departamento[['c_digo_departamento', 'id_departamento']], on='c_digo_departamento', how='left')

display(df_hechos[['id_municipio', 'id_departamento'] + columnas_numericas].head())

,id_municipio,id_departamento,poblaci_n_5_16,tasa_matriculaci_n_5_16,cobertura_neta,cobertura_neta_transici_n,cobertura_neta_primaria,cobertura_neta_secundaria,cobertura_neta_media,cobertura_bruta,cobertura_bruta_transici_n,cobertura_bruta_primaria,cobertura_bruta_secundaria,cobertura_bruta_media,deserci_n,deserci_n_transici_n,deserci_n_primaria,deserci_n_secundaria,deserci_n_media,aprobaci_n,aprobaci_n_transici_n,aprobaci_n_primaria,aprobaci_n_secundaria,aprobaci_n_media,reprobaci_n,reprobaci_n_transici_n,reprobaci_n_primaria,reprobaci_n_secundaria,reprobaci_n_media,repitencia,repitencia_transici_n,repitencia_primaria,repitencia_secundaria,repitencia_media,tama_o_promedio_de_grupo,sedes_conectadas_a_internet
0,1,1,503.0,62.62,62.62,44.19,63.33,51.53,40.23,66.80,58.14,72.86,66.87,56.32,1.19,0.00,1.31,0.00,4.08,92.26,0.00,96.73,83.49,93.88,6.55,0.00,1.96,16.51,2.04,9.52,0.00,10.46,13.76,2.04,23.96,28.13
1,2,2,4438.0,53.27,53.27,33.91,48.89,44.90,21.30,62.98,54.20,65.19,69.60,48.54,5.56,6.95,4.99,6.11,5.26,87.67,0.00,87.90,84.50,92.98,6.78,0.00,7.11,9.39,1.75,9.34,6.95,11.84,8.48,3.16,23.96,28.13
2,3,2,2014.0,32.52,32.52,17.58,25.33,26.43,10.75,38.58,36.36,37.28,46.10,26.16,7.85,15.00,8.43,6.36,4.69,82.68,3.33,84.64,79.51,87.50,9.47,3.33,6.93,14.13,7.81,8.65,6.67,9.04,10.25,1.54,23.96,28.13
3,4,3,10986.0,59.57,59.57,42.76,55.95,43.51,17.06,70.65,64.90,76.96,72.92,53.12,3.95,2.27,1.84,6.77,5.47,90.71,0.57,94.12,84.91,89.93,5.34,0.57,4.04,8.33,4.60,16.18,7.75,21.04,13.84,7.18,23.96,28.13
4,5,3,1228.0,51.30,51.30,76.32,52.29,33.71,11.94,55.54,92.11,65.21,51.12,27.36,8.36,4.29,3.05,15.72,14.55,82.40,0.00,89.63,69.00,78.18,9.24,0.00,7.32,15.28,7.27,9.24,2.86,7.62,14.85,3.64,23.96,28.13


In [75]:
conn = sqlite3.connect('educacion')
cursor = conn.cursor()

In [76]:
df_hechos.to_sql('hechos_educacion', conn, if_exists='replace', index=False)

14582

In [77]:
def ejecutar_sql(query, conexion=conn):
  """
  Toma una consulta SQL, la ejecuta y devuelve el resultado
  en una tabla de pandas para una visualización clara.
  """
  df = pd.read_sql_query(query, conexion)
  # Usamos display() porque en Colab presenta las tablas de forma más elegante.
  display(df)

PREGUNTA 1: ¿Qué porcentaje de escolaridad hay respecto a la población del municipio?

In [78]:
consulta_tasa_matriculacion_municipio = """
    SELECT dm.municipio, 
           he.tasa_matriculaci_n_5_16
    FROM df_hechos AS he
    JOIN dim_municipio AS dm ON he.id_municipio = dm.id_municipio
    ORDER BY he.tasa_matriculaci_n_5_16 DESC;
"""

resultados_tasa_matriculacion_municipio = ejecutar_sql(consulta_tasa_matriculacion_municipio)

print("Tasa de matrícula por municipio (ordenados de mayor a menor):")
print(resultados_tasa_matriculacion_municipio)

,municipio,tasa_matriculaci_n_5_16
0,Puerto Colombia,279.03
1,Puerto Colombia,273.80
2,Puerto Colombia,266.50
3,Cota,240.70
4,Cota,239.34
...,...,...
14577,Papunaua,0.00
14578,Papunaua,0.00
14579,Papunaua,0.00
14580,La Victoria,0.00


Tasa de matrícula por municipio (ordenados de mayor a menor):
None


In [79]:
consulta_esquema = "PRAGMA table_info(hechos_educacion);"
resultados_esquema = ejecutar_sql(consulta_esquema)
print("Esquema de la tabla hechos_educacion:")
print(resultados_esquema)


,cid,name,type,notnull,dflt_value,pk
0,0,c_digo_municipio,TEXT,0,None,0
1,1,c_digo_departamento,TEXT,0,None,0
2,2,a_o,TIMESTAMP,0,None,0
3,3,poblaci_n_5_16,REAL,0,None,0
4,4,tasa_matriculaci_n_5_16,REAL,0,None,0
5,5,cobertura_neta,REAL,0,None,0
6,6,cobertura_neta_transici_n,REAL,0,None,0
7,7,cobertura_neta_primaria,REAL,0,None,0
8,8,cobertura_neta_secundaria,REAL,0,None,0
9,9,cobertura_neta_media,REAL,0,None,0


Esquema de la tabla hechos_educacion:
None


2. ¿Cómo compararía el rendimiento educativo por municipios?

In [ ]:
# Consulta para comparar el rendimiento educativo por municipio
consulta_2 = """
SELECT 
    c_digo_municipio AS codigo_municipio,
    a_o,
    aprobaci_n AS tasa_aprobacion
FROM hechos_educacion
WHERE aprobaci_n IS NOT NULL
ORDER BY c_digo_municipio, a_o;
"""

resultados_rendimiento_educativo = ejecutar_sql(consulta_2)

,codigo_municipio,a_o,tasa_aprobacion
0,05001,2023-01-01 00:00:00,85.37
1,05002,2023-01-01 00:00:00,88.08
2,05004,2023-01-01 00:00:00,92.26
3,05021,2023-01-01 00:00:00,88.86
4,05030,2023-01-01 00:00:00,84.95
...,...,...,...
14577,99773,2019-01-01 00:00:00,70.06
14578,99773,2020-01-01 00:00:00,80.27
14579,99773,2021-01-01 00:00:00,73.55
14580,99773,2022-01-01 00:00:00,72.73


3. ¿Que departamentos son los que mejor cobertura tienen? ¿Pueden hacer cálculo con SQL?

In [87]:
consulta_3 = """
SELECT 
    c_digo_departamento AS codigo_departamento,
    ROUND(AVG(cobertura_neta), 2) AS cobertura_promedio
FROM hechos_educacion
WHERE cobertura_neta IS NOT NULL
GROUP BY c_digo_departamento
ORDER BY cobertura_promedio DESC;
"""

resultados_mejor_cobertura_departamentos = ejecutar_sql(consulta_3)

,codigo_departamento,cobertura_promedio
0,63,94.58
1,70,93.85
2,20,93.73
3,47,93.27
4,50,90.65
5,11,89.49
6,73,89.24
7,25,88.55
8,85,88.53
9,5,88.43
